In [1]:
import sys
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate 
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
q ='''select distinct a.id as advisory_id, a.severity, a.disclose_date, a.publish_date,
        CWE, p.id as package_id, p.name, p.ecosystem, p.repository_url,
        ri.id as release_id, ri.version, ri.prior_release, ri.publish_date as release_date
from advisory a
join package p on a.package_id = p.id
join advisoryCWE aC on a.id = aC.advisory_id
join fixing_releases fr on a.id = fr.advisory_id
join release_info ri on ri.package_id = p.id
    and ri.version = fr.version
where ri.prior_release is not null
and ri.prior_release != 'manual checkup needed'
    and ri.prior_release != 'branch does not match'
    and ri.prior_release != 'not valid semver formatting'
    and repository_url != 'no repository listed' '''
df = pd.DataFrame(sql.execute(q))
df['advisory_id'].nunique(), df['package_id'].nunique(), df['release_id'].nunique()

(3520, 1725, 3856)

In [45]:
q ='''select distinct a.id as advisory_id, a.severity, a.disclose_date, a.publish_date,
        CWE, p.id as package_id, p.name, p.ecosystem, p.repository_url,
        ri.id as release_id, ri.version, ri.prior_release, ri.publish_date as release_date
from advisory a
join package p on a.package_id = p.id
join advisoryCWE aC on a.id = aC.advisory_id
join fixing_releases fr on a.id = fr.advisory_id
join release_info ri on ri.package_id = p.id
    and ri.version = fr.version
where ri.prior_release is not null
and ri.prior_release != 'manual checkup needed'
    and ri.prior_release != 'branch does not match'
    and ri.prior_release != 'not valid semver formatting'
    and repository_url != 'no repository listed'
and (ri.id in (select distinct release_id from version_diff)
or (ri.id in (select release_id from version_diff_with_no_package_file ))
) '''
df = pd.DataFrame(sql.execute(q))
df['advisory_id'].nunique(), df['package_id'].nunique(), df['release_id'].nunique()

(3082, 1542, 3355)

In [51]:
q ='''select *
from advisory a
join package p on a.package_id = p.id
join advisoryCWE aC on a.id = aC.advisory_id
join fixing_releases fr on a.id = fr.advisory_id
join release_info ri on ri.package_id = p.id
    and ri.version = fr.version
join rq3 on rq3.release_id = ri.id
join release_type rt on rt.release_info_id = rq3.release_id
where ri.prior_release is not null
and ri.prior_release != 'manual checkup needed'
    and ri.prior_release != 'branch does not match'
    and ri.prior_release != 'not valid semver formatting'
    and repository_url != 'no repository listed'
    and files_changed > 0 '''
df = pd.DataFrame(sql.execute(q))
df['advisory_id'].nunique(), df['package_id'].nunique(), df['release_id'].nunique()

(2942, 1462, 3153)

In [131]:
q ='''select distinct ecosystem, release_id, rt.type, ri.version
from advisory a
join package p on a.package_id = p.id
join advisoryCWE aC on a.id = aC.advisory_id
join fixing_releases fr on a.id = fr.advisory_id
join release_info ri on ri.package_id = p.id
    and ri.version = fr.version
join rq3 on rq3.release_id = ri.id
join release_type rt on rt.release_info_id = rq3.release_id
where ri.prior_release is not null
and ri.prior_release != 'manual checkup needed'
    and ri.prior_release != 'branch does not match'
    and ri.prior_release != 'not valid semver formatting'
    and repository_url != 'no repository listed'
    and files_changed > 0 '''
rt = pd.DataFrame(sql.execute(q))
total_count = len(rt)
total = rt.groupby('ecosystem')[['release_id']].count()

# patch release
p = rt[(rt['type']=='patch') & (~rt['version'].str.startswith('0'))]
all_p = round((len(p)/total_count) * 100 , 1)
all_p = str(all_p) + '%'
p = p.groupby('ecosystem')[['release_id']].count()
p = reduce(lambda x,y : pd.merge(x,y,on='ecosystem'),[total, p])
p['patch'] = round(p['release_id_y']/p['release_id_x']*100, 1)
p['patch'] = p['patch'].map(str) + '%'


b = rt[(rt['type']=='major') & (~rt['version'].str.startswith('0'))]
all_b = round((len(b)/total_count) * 100 , 1)
all_b = str(all_b) + '%'
b = b.groupby('ecosystem')[['release_id']].count()
b = reduce(lambda x,y : pd.merge(x,y,on='ecosystem'),[total, b])
b['breaking'] = round(b['release_id_y']/b['release_id_x']*100, 1)
b['breaking'] = b['breaking'].map(str) + '%'



p = p.drop(['release_id_y', 'release_id_x'], axis=1)
b = b.drop(['release_id_y', 'release_id_x'], axis=1)

u = rt[rt['version'].str.startswith('0.')]
len(u)

343

In [124]:
All = ['All', df['advisory_id'].nunique(), df['package_id'].nunique(), df['cwe'].nunique(), df['release_id'].nunique(), all_patch  , all_b, df['files_changed'].median(), df['loc_changed'].median()]
All

['All', 2942, 1462, 152, 3153, '66.4%', '13.4%', 10.0, 195.0]

In [128]:
adv = df.groupby('ecosystem')[['advisory_id']].nunique()
pkg = df.groupby('ecosystem')[['package_id']].nunique()
rel = df.groupby('ecosystem')[['release_id']].nunique()
cwe = df.groupby('ecosystem')[['cwe']].nunique()
file = df.groupby('ecosystem')[['files_changed']].median()
loc = df.groupby('ecosystem')[['loc_changed']].median()
final = reduce(lambda x,y : pd.merge(x,y,on='ecosystem'),[adv, pkg, cwe, rel, p, b, file, loc])

final = final.reset_index()
final.loc[len(final)] = All
final


,ecosystem,advisory_id,package_id,cwe,release_id,patch,breaking,files_changed,loc_changed
0,Composer,742,198,63,784,85.3%,2.2%,25.0,429.5
1,Go,147,111,42,166,61.4%,25.3%,3.0,69.5
2,Maven,589,293,77,754,66.8%,4.2%,13.0,310.0
3,NuGet,190,44,20,110,87.3%,3.6%,25.0,507.0
4,RubyGems,203,112,46,265,64.5%,16.6%,4.0,40.0
5,npm,690,486,79,734,54.6%,26.0%,3.0,50.0
6,pip,381,218,68,340,44.7%,27.1%,7.0,120.0
7,All,2942,1462,152,3153,66.4%,13.4%,10.0,195.0


In [129]:
print(tabulate(final, tablefmt='latex', headers='keys',showindex=False))

\begin{tabular}{lrrrrllrr}
\hline
 ecosystem   &   advisory\_id &   package\_id &   cwe &   release\_id & patch   & breaking   &   files\_changed &   loc\_changed \\
\hline
 Composer    &           742 &          198 &    63 &          784 & 85.3\%   & 2.2\%       &              25 &         429.5 \\
 Go          &           147 &          111 &    42 &          166 & 61.4\%   & 25.3\%      &               3 &          69.5 \\
 Maven       &           589 &          293 &    77 &          754 & 66.8\%   & 4.2\%       &              13 &         310   \\
 NuGet       &           190 &           44 &    20 &          110 & 87.3\%   & 3.6\%       &              25 &         507   \\
 RubyGems    &           203 &          112 &    46 &          265 & 64.5\%   & 16.6\%      &               4 &          40   \\
 npm         &           690 &          486 &    79 &          734 & 54.6\%   & 26.0\%      &               3 &          50   \\
 pip         &           381 &          218 &    68 &

In [42]:
q ='''select  p.ecosystem, 
        ri.id as release_id, rq3.*
from advisory a
join package p on a.package_id = p.id
join advisoryCWE aC on a.id = aC.advisory_id
join fixing_releases fr on a.id = fr.advisory_id
join release_info ri on ri.package_id = p.id
    and ri.version = fr.version
join rq3 on rq3.release_id = ri.id
where ri.prior_release is not null
and ri.prior_release != 'manual checkup needed'
    and ri.prior_release != 'branch does not match'
    and ri.prior_release != 'not valid semver formatting'
    and repository_url != 'no repository listed'
    and files_changed > 0 '''
df = pd.DataFrame(sql.execute(q))

In [41]:
df['loc_changed'].median()

176.0

In [52]:
df.groupby('ecosystem')[['loc_changed']].median()

,loc_changed
ecosystem,
Composer,429.5
Go,69.5
Maven,310.0
NuGet,507.0
RubyGems,40.0
npm,50.0
pip,120.0


In [53]:
df.groupby('ecosystem')[['files_changed']].median()

,files_changed
ecosystem,
Composer,25
Go,3
Maven,13
NuGet,25
RubyGems,4
npm,3
pip,7


In [28]:
random = df.sample(frac=1).reset_index(drop=True)
random.to_csv('rq3_random.csv', index=False)

In [18]:
len(random)

4582

In [ ]:
q ='''select distinct a.id as advisory_id, a.severity, a.disclose_date, a.publish_date,
        CWE, p.id as package_id, p.name, p.ecosystem, p.repository_url,
        ri.id as release_id, ri.version, ri.prior_release, ri.publish_date as release_date
from advisory a
join package p on a.package_id = p.id
join advisoryCWE aC on a.id = aC.advisory_id
join fixing_releases fr on a.id = fr.advisory_id
join release_info ri on ri.package_id = p.id
    and ri.version = fr.version
join version_diff vd on vd.release_id = ri.id
where ri.prior_release is not null
and ri.prior_release != 'manual checkup needed'
    and ri.prior_release != 'branch does not match'
    and ri.prior_release != 'not valid semver formatting'
    and repository_url != 'no repository listed' '''
df = pd.DataFrame(sql.execute(q))
df['advisory_id'].nunique(), df['package_id'].nunique(), df['release_id'].nunique()

83173.0